In [1]:
from numpy.random import binomial
#probability 0< f <= 0.5
f = 0.3
#number of neurons
N = 10
#number of random patterns
p = 7

def generate_data():
    eta_i_mu = binomial(1,f, (p,N))
    return(eta_i_mu)
    
    

In [7]:
# https://blog.dbrgn.ch/2013/3/26/perceptrons-in-python/
# http://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/

from random import choice
from numpy import array, dot, random


#Decision boundary
T = 0
#Robustness
K = 0

# Unit_step does not include K, the robustness. 
# A preprocessing step will ignore data too close to the boundary.
unit_step = lambda x: 0 if x < T else 1

#No dummy, because the decision boundary stays in the unit_step function
training_data = generate_data()

w = random.rand(N,N)
errors = []
l_rate = 0.2
n = 100
for mu in range(training_data.shape[0]):
    x = training_data[mu,:]
    for i in range(training_data.shape[0]):
        expected = training_data[mu,i]
        wi = w[i,:]
        result = dot(wi, x)
        if (result > T+K) or (result < T-K):
            error = expected - unit_step(result)
        else:
            error = 1 if expected == 1 else -1
        errors.append(error)
        wi += l_rate * error * x
        #Non-negativity constraint
        wi = wi>0 * wi

for mu in range(training_data.shape[0]):
    x = training_data[mu,:]
    for i in range(training_data.shape[0]):
        expected = training_data[mu,i]
        wi = w[i,:]
        result = dot(x, wi)
        print("{}: {} -> {}".format(expected, result, unit_step(result)))

1: 0.5048182454292507 -> 1
0: -0.8063718732525773 -> 0
0: 0.15071620286686388 -> 1
0: 0.27297637256607893 -> 1
0: -0.17702872056458024 -> 0
0: -0.342971700467114 -> 0
0: -0.16941832532113 -> 0
1: 1.1642656274839478 -> 1
0: -0.5872122383907095 -> 0
0: 0.851503085211425 -> 1
0: -0.20786284584590559 -> 0
0: -0.11434896826924645 -> 0
0: 0.26093183533967285 -> 1
0: 0.14979045720113104 -> 1
0: -0.003830924278800607 -> 0
0: -0.5130614196071198 -> 0
0: 0.0302120310467629 -> 1
0: 0.33424629539972933 -> 1
0: -0.09306178832069628 -> 0
0: 0.04858146545024178 -> 1
1: 0.9010779974324901 -> 1
1: 0.5443879541749606 -> 1
0: -1.0586183470662376 -> 0
0: 0.3951240592441222 -> 1
1: 0.49725644693999466 -> 1
0: 0.4374277818275047 -> 1
0: 0.5748557291745393 -> 1
1: 1.177590708804898 -> 1
0: 0.8974514045305169 -> 1
0: 0.2931640887798018 -> 1
1: 1.3054638745478448 -> 1
0: -0.3793332169834036 -> 0
0: 0.43025678871593664 -> 1
0: 0.29436792454940275 -> 1
0: 0.4785945564578447 -> 1
0: -0.21532471076064685 -> 0
0: 0